In [ ]:
!pip install scikit-learn
import sklearn
print(sklearn.__version__)  
from sklearn.metrics.pairwise import cosine_similarity


customer_features_normalized = customer_features.drop(columns=['CustomerID'])


similarity_matrix = cosine_similarity(customer_features_normalized)


similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])


similarity_df.head()


In [ ]:
def get_top_3_similar_customers(customer_id):
    similarity_scores = similarity_df[customer_id]
    
   
    similar_customers = similarity_scores.drop(customer_id).sort_values(ascending=False).head(3)
    
    
    return similar_customers.index.tolist(), similar_customers.values.tolist()


top_3_customers, similarity_scores = get_top_3_similar_customers('C0001')


print(f"Top 3 similar customers for C0001: {top_3_customers} with similarity scores: {similarity_scores}")


In [ ]:

recommendations = []


for customer_id in customer_features['CustomerID']:
    similar_customers, similarity_scores = get_top_3_similar_customers(customer_id)
    for rec_customer, score in zip(similar_customers, similarity_scores):
        recommendations.append([customer_id, rec_customer, score])


lookalike_df = pd.DataFrame(recommendations, columns=['CustomerID', 'RecommendedCustomerID', 'SimilarityScore'])


lookalike_df.to_csv('Lookalike.csv', index=False)


lookalike_df.head()


In [ ]:
lookalike_df.to_csv('Lookalike.csv', index=False)


In [ ]:
from IPython.display import FileLink


FileLink('Lookalike.csv')


In [ ]:
customer_data = pd.merge(transactions, customers, on='CustomerID', how='inner')
customer_data.head()



In [ ]:
customer_data['TransactionDate'] = pd.to_datetime(customer_data['TransactionDate'])

total_spending = customer_data.groupby('CustomerID')['TotalValue'].sum()
num_transactions = customer_data.groupby('CustomerID')['TransactionID'].count()
avg_transaction_value = customer_data.groupby('CustomerID')['TotalValue'].mean()
last_transaction_date = customer_data.groupby('CustomerID')['TransactionDate'].max()
recency = (customer_data['TransactionDate'].max() - last_transaction_date).dt.days
first_transaction_date = customer_data.groupby('CustomerID')['TransactionDate'].min()
frequency = (last_transaction_date - first_transaction_date).dt.days

customer_features = pd.DataFrame({
    'TotalSpending': total_spending,
    'NumTransactions': num_transactions,
    'AvgTransactionValue': avg_transaction_value,
    'Recency': recency,
    'Frequency': frequency
})

customer_features.head()
